In [28]:
# ==== Setup & Imports  ====

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype

pd.set_option("display.max_rows", 150)
pd.set_option("display.max_columns", 150)

print("Ready")


Ready


In [33]:
# Load dataset directly from Colab working directory
RAW = "/content/drive/MyDrive/heartAttack_dirty(2).csv"   # file already visible in left panel
df = pd.read_csv(RAW)

rows, cols = df.shape
print(f"Shape: {rows} rows x {cols} cols")

# Identify target column
target = next((c for c in df.columns if c.lower().replace(" ", "_")=="heart_attack"), None)
print("Target column:", target)



Shape: 120259 rows x 28 cols
Target column: heart_attack


In [34]:
def clean_member1_strict(df: pd.DataFrame, target_col: str|None, rare_thresh_ratio: float = 0.005):
    """
    - Trim/lower object columns (skip id/name/email, protect target)
    - Normalize boolean-like
    - Convert numeric-like strings
    - Guard ranges for clinical columns (incl. total cholesterol)
    - Fix BP inversions (swap dia/sys if needed)
    - Negative & zero handling for key labs
    - Impute: numeric -> median, categorical -> 'unknown'
    - Drop missing target, drop duplicates
    - Optional rare-category bucketing: categories < thresh -> 'other'
    """
    df = df.copy()

    # 0) helpers
    def norm_name(s: str) -> str:
        return s.lower().replace(" ", "_")
    cols_norm = {c: norm_name(c) for c in df.columns}

    # 1) Trim & normalize categoricals
    obj_cols = list(df.select_dtypes(include=["object"]).columns)
    for c in obj_cols:
        s = df[c].astype("string").str.strip()
        if c != target_col and not any(k in c.lower() for k in ["name","email","id"]):
            df[c] = s.str.lower()
        else:
            df[c] = s

    # 2) Normalize boolean-like to yes/no (skip target)
    truthy = {"yes","y","true","t","1","positive","pos","present"}
    falsy  = {"no","n","false","f","0","negative","neg","absent"}
    boollike_cols = []
    for c in obj_cols:
        if c == target_col:
            continue
        vals = set(df[c].dropna().unique().tolist())
        if vals and len(vals) <= 6:
            ratio = sum(v in truthy or v in falsy for v in vals) / len(vals)
            if ratio > 0.8:
                df[c] = df[c].map(lambda v: ("yes" if v in truthy else ("no" if v in falsy else v)) if pd.notna(v) else v)
                boollike_cols.append(c)

    # 3) Convert numeric-like objects (skip target)
    converted_cols = []
    for c in obj_cols:
        if c == target_col:
            continue
        co = pd.to_numeric(df[c].astype(str).str.replace(",","").str.replace(" ",""), errors="coerce")
        if co.notna().mean() > 0.9:
            df[c] = co
            converted_cols.append(c)

    # 4) Clinical guards
    # 4a) Define ranges
    rules = {
        "age": (0, 120),
        "sleep_hours": (0, 24),
        "systolic_bp": (60, 260),
        "diastolic_bp": (30, 160),
        "hdl": (5, 150),
        "ldl": (20, 400),
        "triglycerides": (20, 1500),
        "bmi": (10, 80),
        "fasting_blood_sugar": (40, 400),
    }

    # 4b) Add any TOTAL cholesterol column (exclude hdl/ldl)
    chol_like = []
    for c in df.columns:
        lc = cols_norm[c]
        if ("cholesterol" in lc) and ("hdl" not in lc) and ("ldl" not in lc):
            rules[lc] = (50, 500)  # conservative; catches negatives/absurd highs
            chol_like.append(c)

    applied_rules = []
    for name, (lo, hi) in rules.items():
        matches = [c for c in df.columns if name in cols_norm[c]]
        for c in matches:
            if c != target_col and is_numeric_dtype(df[c]):
                s = df[c]
                # Treat negatives as NaN
                s = s.where((s >= 0) | s.isna())
                # Treat zeros as NaN for labs where zero is impossible
                if any(k in name for k in ["bp","hdl","ldl","triglycerides","bmi","fasting_blood_sugar","cholesterol"]):
                    s = s.where((s != 0) | s.isna())
                # Apply range bounds
                df[c] = s.where(s.between(lo,hi) | s.isna())
                applied_rules.append(c)

    # 4c) Blood pressure sanity: swap if diastolic > systolic
    # Try to find columns
    sys_cols = [c for c in df.columns if "systolic" in cols_norm[c] or cols_norm[c].endswith("_sbp")]
    dia_cols = [c for c in df.columns if "diastolic" in cols_norm[c] or cols_norm[c].endswith("_dbp")]
    if sys_cols and dia_cols:
        # assume one each if multiple found use first (or extend for all pairs)
        s = sys_cols[0]; d = dia_cols[0]
        if is_numeric_dtype(df[s]) and is_numeric_dtype(df[d]):
            mask_swap = (df[d].notna() & df[s].notna() & (df[d] > df[s]))
            if mask_swap.any():
                s_tmp = df.loc[mask_swap, s].copy()
                df.loc[mask_swap, s] = df.loc[mask_swap, d]
                df.loc[mask_swap, d] = s_tmp

    # 5) Optional: rare-category bucketing (for high-cardinality categoricals)
    # Any category with frequency < rare_thresh_ratio gets 'other'
    rare_info = {}
    for c in df.select_dtypes(include=["object","string"]).columns:
        if c == target_col:
            continue
        vc = df[c].value_counts(dropna=False, normalize=True)
        rare = vc[vc < rare_thresh_ratio].index
        if len(rare) > 0 and len(vc) > 10:  # avoid tiny cols
            df[c] = df[c].where(~df[c].isin(rare), other="other")
            rare_info[c] = int(len(rare))

    # 6) Impute missing (features only)
    for c in df.columns:
        if c == target_col:
            continue
        if df[c].dtype == "object" or str(df[c].dtype).startswith("string"):
            df[c] = df[c].fillna("unknown")
        elif is_numeric_dtype(df[c]) and df[c].isna().any():
            df[c] = df[c].fillna(df[c].median())

    # 7) Drop rows with missing target (never impute)
    dropped_missing_target = 0
    if target_col and df[target_col].isna().any():
        before = len(df)
        df = df[df[target_col].notna()].copy()
        dropped_missing_target = before - len(df)

    # 8) Remove exact duplicates
    before = len(df)
    df = df.drop_duplicates()
    dropped_dups = before - len(df)

    log = {
        "target_column": target_col,
        "dropped_missing_target_rows": int(dropped_missing_target),
        "exact_duplicates_dropped": int(dropped_dups),
        "object_to_numeric_converted": converted_cols,
        "boolean_like_normalized": boollike_cols,
        "range_rules_applied_on_columns": sorted(set(applied_rules)),
        "cholesterol_like_columns": chol_like,
        "rare_bucketed_columns": rare_info,  # col -> #rare categories grouped
        "final_shape": (len(df), df.shape[1]),
    }
    return df, log


In [35]:
# Run
clean, log = clean_member1_strict(df, target)
print("LOG:\n", log)

# Verification: ensure no negative cholesterols remain
chol_cols = [c for c in clean.columns if "cholesterol" in c.lower()]
for c in chol_cols:
    if is_numeric_dtype(clean[c]):
        assert (clean[c] < 0).sum() == 0, f"Negative values remain in {c}"

# Quick after-audit for your report
audit_after = pd.DataFrame({
    "column": clean.columns,
    "dtype": [str(t) for t in clean.dtypes],
    "n_unique": [clean[c].nunique() for c in clean.columns],
    "missing": [clean[c].isna().sum() for c in clean.columns],
})
audit_after["missing_%"] = (audit_after["missing"] / len(clean) * 100).round(2)

# Save
clean_path = "heartAttack_cleaned.csv"
audit_path = "heartattack_audit_summary.csv"
log_path = "heartattack_cleaning_log.txt"

clean.to_csv(clean_path, index=False)
audit_after.to_csv(audit_path, index=False)
with open(log_path, "w") as f:
    for k,v in log.items():
        f.write(f"{k}: {v}\n")

print("Saved:", clean_path, audit_path, log_path)

# Download to your machine
from google.colab import files
files.download(clean_path)
files.download(audit_path)
files.download(log_path)


LOG:
 {'target_column': 'heart_attack', 'dropped_missing_target_rows': 3, 'exact_duplicates_dropped': 1, 'object_to_numeric_converted': [], 'boolean_like_normalized': [], 'range_rules_applied_on_columns': ['age', 'cholesterol_hdl', 'cholesterol_ldl', 'cholesterol_level', 'fasting_blood_sugar', 'medication_usage', 'sleep_hours', 'triglycerides'], 'cholesterol_like_columns': ['cholesterol_level'], 'rare_bucketed_columns': {}, 'final_shape': (120255, 28)}
Saved: heartAttack_cleaned.csv heartattack_audit_summary.csv heartattack_cleaning_log.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>